<a href="https://colab.research.google.com/github/taolaobd/CS114.L22.KHCL/blob/main/Assignments/Assignments6/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Máy học - CS114.L22.KHCL 
#BÀI TẬP COLLAB SỐ 5: DỰ ĐOÁN TIÊU ĐỀ CHÂM BIẾM

##Thành viên: 
1.	Hồ Hải Thủy	19522323		

2. Nguyễn Mạnh Toàn	19522363	

3. Nguyễn Khả Tiến	19522337		
	

# Thư viện hỗ trợ Word2Vec

In [ ]:
!pip install nltk
!pip install gensim

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import gensim
from gensim.models import Word2Vec
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Nếu cảnh báo lỗi, mở dòng này
import warnings
warnings.filterwarnings(action = 'ignore')

# Tiền xử lý dữ liệu

Download data được thu thập từ trước và nén thành file zip để sẵn trong drive.

> Indented block



In [ ]:
!gdown --id 11JjlsjDiElNdCpggyar7xP6XITgw0dLW
!unzip text.zip

Downloading...
From: https://drive.google.com/uc?id=11JjlsjDiElNdCpggyar7xP6XITgw0dLW
To: /content/text.zip
2.20MB [00:00, 70.6MB/s]
Archive:  text.zip
  inflating: bestinau.json           
  inflating: betootaadvocate.json    
  inflating: chaser.json             
  inflating: clickhole.json          
  inflating: thecivilian.json        
  inflating: thehardtimes.json       
  inflating: thepoke.json            
  inflating: time.json               
  inflating: toronto.json            


Để tiện xử lý, lấy trước tên các bài báo trong file zip

In [ ]:
# Lấy tên các file trong file zip

import zipfile
list_name = zipfile.ZipFile('text.zip').namelist()
list_name

['bestinau.json',
 'betootaadvocate.json',
 'chaser.json',
 'clickhole.json',
 'thecivilian.json',
 'thehardtimes.json',
 'thepoke.json',
 'time.json',
 'toronto.json']

In [ ]:
pd.read_json(list_name[2])

,article_link,headline,is_sarcastic
0,https://chaser.com.au/national/local-mp-only-g...,\nLocal MP only got into politics so they coul...,1
1,https://chaser.com.au/national/god-distances-h...,\nGod distances himself from Scott Morrison af...,1
2,https://chaser.com.au/national/angus-taylor-ta...,\nAngus Taylor takes break from dodgy Federal ...,1
3,https://chaser.com.au/general-news/legally-you...,\n“Legally you have to tell me if you’re a cop...,1
4,https://chaser.com.au/national/theres-no-money...,\n“There’s no money to help arts workers” says...,1
...,...,...,...
1794,https://chaser.com.au/national/australian-publ...,\nAustralian public outraged by Milo ban until...,1
1795,https://chaser.com.au/live/,\nLive redirect,1
1796,https://chaser.com.au/general-news/police-call...,\nPolice call for witnesses after a local resi...,1
1797,https://chaser.com.au/page/148/?s,None,1


Xem thử trước data của 1 trang báo ở trên. Ta thấy còn các giá trị None cần loại bỏ.


Trong quá trình train thử trong Word2Vec thì đã có lỗi, kiểm tra thấy headline lỗi là một headline không có ký tự, chỉ gồm dấu xuống dòng, cần loại bỏ luôn.

In [ ]:
#Đọc từng file báo

Article = []
for i in list_name:
  a = pd.read_json(i)
  a = a.dropna()
  a = a.drop(a.index[a['headline']=='\n'].tolist())
  Article.append([a])

Article là 1 list các trang báo. Mỗi phần tử sẽ là 1 list khác, có phần từ đầu là dataframe của bài báo và tiếp theo là các Word2Vec đại diện cho từng câu headline trong bài báo.

Input của Word2Vec yêu cầu là một list các từ trong một câu headline nên cần có hàm chuyển đổi.

In [ ]:
#Hàm chuyển 1 câu headline thành 1 list các từ 

def Convert_str(f):
  f = f.replace("\n", " ")
  data = []  
  # Duyệt từng câu trong tệp
  for i in sent_tokenize(f):
    temp = []
    # Chuyển câu thành các từ
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)
  return data

Tạo thêm hàm để xuất các vector biểu diễn của các từ trong 1 headline kèm với từ nó biểu diễn để dễ theo dõi.

In [ ]:
#Xuất ra vector biểu diễn của các từ trong 1 headline

def read_word2vec(a):
  wordvector = list(a.wv.vocab)
  print(wordvector)
  for i in wordvector:
    print(i, a[i])

## Word Embedding
- Có 2 mô hình triển khai là CBOW và Skip Gram. Mặc định là CBOW, thêm tham số sg = 1 là Skip Gram.
- min_count là số lần xuất hiện, mặc định là 5 nên sẽ gặp lỗi nều chuỗi không có từ nào lặp lại 5 lần
- size là kích thước vector đầu ra
- window là số lượng từ ngữ cảnh bổ nghĩa xung quanh được lấy

Dưới là sử dụng mô hình dự đoán CBOW. Mô hình này có Input: là từ ngữ cảnh, Ouput: Từ hiện tại. Skip Gram ngược lại.

Tiến hành train từng headline của mỗi bài báo. Vì headline của một bài báo thường ngắn, nên số từ lặp lại sẽ rất ít, do vậy tham số min_count sẽ bằng 1.

In [ ]:
#Chuyển các headline về dưới dạng vector
for i in Article:
  #Lấy phần HeadLine:
  for j in i[0]['headline']:
    WordEmbedding = Word2Vec(Convert_str(j), min_count=1, window = 5, size = 100)
    i.append(WordEmbedding)

Đọc thử các headline đầu tiên của mỗi trang báo

In [ ]:
for i in Article:
  read_word2vec(i[1])
  print()

['9', 'travel', 'trends', 'that', 'are', 'here', 'to', 'stay']
9 [ 4.2326995e-03  4.9317465e-03 -9.9836243e-04 -1.0781001e-03
  3.8025859e-03 -9.6436305e-04  1.2471826e-03  3.8764637e-03
  1.1364747e-03 -2.6412732e-03 -3.2981662e-03  1.5089125e-03
  3.3423929e-03 -6.7658396e-04 -2.7538741e-03 -1.5426256e-03
 -4.9769727e-04 -2.3090478e-03 -1.3448395e-03  2.5648023e-03
  2.7761001e-03  2.2312941e-03  3.0022352e-03  3.5360935e-03
 -4.9291505e-03  2.8868674e-03 -6.3481653e-04 -1.0797379e-03
 -3.2933324e-03  2.1932023e-03  4.9107638e-03 -7.4496306e-04
  3.8504242e-03 -1.3893191e-03  8.6312363e-04  8.2742836e-04
 -1.7767679e-03 -2.8425665e-03  6.5765233e-04  1.0766590e-03
 -3.8034047e-03  3.9810464e-03 -3.1736284e-04 -4.0077893e-03
  3.0726402e-03  6.4862729e-04 -2.2631946e-03 -1.5561904e-03
  2.0997592e-03  1.4630387e-03  4.3347813e-03 -2.9878016e-03
 -8.9566811e-04  3.0662271e-03  2.7494209e-03 -3.0936389e-03
 -3.7137081e-03  2.9147179e-03  2.1458843e-03 -1.0037214e-03
  3.4058853e-03  4.3

Ta thấy Word2Vec trả về là 1 mảng vector của mỗi từ. Nên muốn lấy vector đại diện cho 1 câu thì ta tính trung bình của mỗi từ trong câu.

In [ ]:
#Hàm nhận vào 1 Word2Vec và trả về 1 vector đại diện
import numpy as np
def get_embedding_w2v(doc_tokens):
    embeddings = []
    for tok in doc_tokens.wv.vocab:
      embeddings.append(doc_tokens.wv.word_vec(tok))
    return np.mean(embeddings, axis=0)

Chia data để train, X là các features, Y là kết quả dự đoán

In [ ]:
X = []
for i in Article:
  for j in range(1, len(i)):
    X.append(get_embedding_w2v(i[j]))

In [ ]:
Y = []
for i in Article:
  Y += list(i[0]['is_sarcastic'])

# Train Model lần 1

Import 1 số model để thử nghiệm

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 5)

Model 1: Linear support vector classifier

In [ ]:
lsvc = LinearSVC()
# Train model
lsvc.fit(X_train, Y_train)
Y_pred1 = lsvc.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred1)


print(lsvc.score(X_train, Y_train)) 
print(lsvc.score(X_test, Y_test))   

0.6559519364583687
0.6476019845644984


Model 2: Gaussuan Naive Bayes

In [ ]:
gnb = GaussianNB()
# Train model
gnb.fit(X_train, Y_train)
Y_pred2 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred2)

print(gnb.score(X_train, Y_train)) 
print(gnb.score(X_test, Y_test))   

0.738365975357252
0.736287210584344


Model 3: Logistic Regression

In [ ]:
lr = LogisticRegression()
# Train model
lr.fit(X_train, Y_train)
Y_pred3 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred3)

print(lr.score(X_train, Y_train)) 
print(lr.score(X_test, Y_test))     

0.5254404127490581
0.5210859977949284


 Model 4: Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
# Train model
rfc.fit(X_train, Y_train)
Y_pred4 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred4)

print(rfc.score(X_train, Y_train)) 
print(rfc.score(X_test, Y_test))    

0.9940599436543227
0.7632304299889746


#Train Model lần 2

Ở lần train model thứ 2, em thêm bước chuẩn hóa dữ liệu để xem liệu độ chính xác có tăng lên không

In [ ]:
from sklearn import preprocessing

X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

Model 1: Linear support vector classifier

In [ ]:
lsvc = LinearSVC()
# Train model
lsvc.fit(X_train, Y_train)
Y2_pred1 = lsvc.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y2_pred1)


print(lsvc.score(X_train, Y_train)) 
print(lsvc.score(X_test, Y_test))   

0.7398255320593327
0.7333241455347299


Model 2: Gaussuan Naive Bayes

In [ ]:
gnb = GaussianNB()
# Train model
gnb.fit(X_train, Y_train)
Y2_pred2 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y2_pred2)

print(gnb.score(X_train, Y_train)) 
print(gnb.score(X_test, Y_test))   

0.738365975357252
0.7364939360529217


Model 3: Logistic Regression

In [ ]:
lr = LogisticRegression()
# Train model
lr.fit(X_train, Y_train)
Y3_pred3 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y3_pred3)

print(lr.score(X_train, Y_train)) 
print(lr.score(X_test, Y_test)) 

0.7389090662231425
0.733255237045204


 Model 4: Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
# Train model
rfc.fit(X_train, Y_train)
Y4_pred4 = gnb.predict(X_test)
mse = sklearn.metrics.mean_squared_error(Y_test, Y4_pred4)

print(rfc.score(X_train, Y_train)) 
print(rfc.score(X_test, Y_test))    

0.9934489664301959
0.7645396912899669
